In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/curated/models_summary.csv')

In [4]:
len(df)

36

Let´s review the accuracy of knn models

In [6]:
df[df['model'] == 'knn']

,Unnamed: 0,accuracy_test,before_data,elapsed_time,model,model_type,params,score,sweep_buy_sell,sweep_news_agg,sweeps_market_variables
0,0,0.324303,excluded,0.51,knn,clf,{'n_neighbors': 31},0.30,15_60,ALL_NO_3_2_1,market_basic
6,0,0.349071,excluded,0.71,knn,clf,{'n_neighbors': 21},0.35,30_60,ALL_NO_3_2_1,market_basic
12,0,0.328947,excluded,0.70,knn,clf,{'n_neighbors': 41},0.32,30_90,ALL_NO_3_2_1,market_basic
18,0,0.315789,excluded,0.77,knn,clf,{'n_neighbors': 11},0.30,30_120,ALL_NO_3_2_1,market_basic
24,0,0.400155,excluded,1.05,knn,clf,{'n_neighbors': 11},0.38,60_120,ALL_NO_3_2_1,market_basic
30,0,0.376161,excluded,0.58,knn,clf,{'n_neighbors': 41},0.37,60_240,ALL_NO_3_2_1,market_basic


Let´s review the accuracy of decision tree models

In [8]:
df[df['model'] == 'tree']

,Unnamed: 0,accuracy_test,before_data,elapsed_time,model,model_type,params,score,sweep_buy_sell,sweep_news_agg,sweeps_market_variables
1,1,0.386223,excluded,0.19,tree,clf,"{'max_depth': 3, 'min_samples_leaf': 10}",0.39,15_60,ALL_NO_3_2_1,market_basic
7,1,0.469040,excluded,0.32,tree,clf,"{'max_depth': 5, 'min_samples_leaf': 100}",0.48,30_60,ALL_NO_3_2_1,market_basic
13,1,0.424149,excluded,0.23,tree,clf,"{'max_depth': 4, 'min_samples_leaf': 100}",0.43,30_90,ALL_NO_3_2_1,market_basic
19,1,0.417957,excluded,0.24,tree,clf,"{'max_depth': 5, 'min_samples_leaf': 100}",0.40,30_120,ALL_NO_3_2_1,market_basic
25,1,0.501548,excluded,0.29,tree,clf,"{'max_depth': 3, 'min_samples_leaf': 10}",0.51,60_120,ALL_NO_3_2_1,market_basic
31,1,0.450464,excluded,0.19,tree,clf,"{'max_depth': 6, 'min_samples_leaf': 200}",0.44,60_240,ALL_NO_3_2_1,market_basic


In [14]:
df.sort_values(by=['accuracy_test'], ascending = False)

,Unnamed: 0,accuracy_test,before_data,elapsed_time,model,model_type,params,score,sweep_buy_sell,sweep_news_agg,sweeps_market_variables
26,2,0.540248,excluded,11.21,forest,clf,"{'max_depth': 5, 'min_samples_leaf': 100, 'n_e...",0.52,60_120,ALL_NO_3_2_1,market_basic
27,3,0.534830,excluded,1.37,xgb,clf,{'n_estimators': 50},0.51,60_120,ALL_NO_3_2_1,market_basic
28,4,0.527090,excluded,1.81,gradientBoosting,clf,{'n_estimators': 50},0.51,60_120,ALL_NO_3_2_1,market_basic
29,5,0.525542,excluded,0.36,AdaBoostClassifier,clf,{'n_estimators': 50},0.48,60_120,ALL_NO_3_2_1,market_basic
8,2,0.506966,excluded,10.98,forest,clf,"{'max_depth': 5, 'min_samples_leaf': 10, 'n_es...",0.49,30_60,ALL_NO_3_2_1,market_basic
25,1,0.501548,excluded,0.29,tree,clf,"{'max_depth': 3, 'min_samples_leaf': 10}",0.51,60_120,ALL_NO_3_2_1,market_basic
9,3,0.493808,excluded,1.79,xgb,clf,{'n_estimators': 100},0.49,30_60,ALL_NO_3_2_1,market_basic
10,4,0.493034,excluded,2.32,gradientBoosting,clf,{'n_estimators': 10},0.48,30_60,ALL_NO_3_2_1,market_basic
11,5,0.475232,excluded,0.48,AdaBoostClassifier,clf,{'n_estimators': 10},0.45,30_60,ALL_NO_3_2_1,market_basic
33,3,0.472910,excluded,1.38,xgb,clf,{'n_estimators': 50},0.44,60_240,ALL_NO_3_2_1,market_basic
